In [6]:
import pandas as pd
import sqlite3
DB_PATH = "alchemy.db"
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

all_effects_df = pd.read_pickle("all_effects_df.pkl")


In [7]:
import pandas as pd
young_alchemy_data = pd.read_csv('young_alchemy.csv')
old_cols = young_alchemy_data.columns
rename_map = {}
_new_cols = ["title","material_analog","main_effect","side_effect","identifier"]
for old_col,new_col in zip(old_cols,_new_cols):
    rename_map[old_col] = new_col

young_alchemy_data.rename(columns=rename_map,inplace=True)
types = ["Магические Металлы","Магические Компоненты","Травы"]
# filtered_young_alchemy_data =young_alchemy_data[~young_alchemy_data['Мат. эквивалент'].apply(lambda x: isinstance(x,str) and any(t in x for t in types))]
# filtered_young_alchemy_data=filtered_young_alchemy_data.apply(lambda x: x.strip if isinstance(x,str) else x)


In [8]:
young_alchemy_data

,title,material_analog,main_effect,side_effect,identifier
0,NaN,Магические Металлы,NaN,NaN,NaN
1,Философская Сера (Разум/Зелёный),Мускатный орех молотый,"Совершение плохого поступка (одно действие, ко...",Против симптомов магических вирусов (откатывае...,FS
2,NaN,NaN,NaN,Вызывает немоту,NaN
3,NaN,NaN,NaN,Средний яд,NaN
4,Философская Ртуть (Время/Оранжевый),Корень солодки (раствор),"Головокружение, проблемы с вестибулярным аппар...",Смертельный яд,FR
...,...,...,...,...,...
151,NaN,NaN,NaN,Эйфория,NaN
152,NaN,NaN,NaN,Невосприимчивость к привлекательности противоп...,NaN
153,Можжевельник кладбищенский,Можжевельник,В составе эликсира дает магу защиту от различн...,Улучшает ментальную защиту,MHK
154,NaN,NaN,NaN,Усиливает любой эликсир/зелье на выходе. Сила ...,NaN


In [9]:
current_material_type = None
current_ingredient = None
current_ingredient_id = None
current_ingredient_code = None
for index,row in young_alchemy_data.iterrows():
    if isinstance(row["material_analog"],str):
        material_analog = row["material_analog"].strip()
        if any(t in material_analog for t in types):
            current_material_type = material_analog
            # young_alchemy_data.at[index,"material_analog"] = None
    if isinstance(row["title"],str):
        current_ingredient = row["title"].strip()
        main_effect = row["main_effect"].strip()
        side_effect = row["side_effect"].strip()
        identifier = row["identifier"].strip()
        material_analog = row["material_analog"].strip()

        cursor.execute("""
            SELECT id FROM ingredients WHERE code = ?
            """,(identifier,))
        current_ingredient_id = cursor.fetchone()
        if current_ingredient_id:
            current_ingredient_id = current_ingredient_id[0]
            current_ingredient=None
        else:
            cursor.execute("""
                INSERT INTO ingredients (code,material_analog,ingredient_type,name) VALUES (?,?,?,?)
                """,(identifier,material_analog,current_material_type,current_ingredient))
            conn.commit()
            current_ingredient_id = cursor.lastrowid

            cursor.execute("""
                SELECT id FROM effects WHERE description = ?
                """,(main_effect.lower(),))
            main_effect_id = cursor.fetchone()
            if main_effect_id:
                main_effect_id = main_effect_id[0]
            else:
                cursor.execute("""
                    INSERT INTO effects (description) VALUES (?)
                    """,(main_effect.lower(),))
                conn.commit()
                main_effect_id = cursor.lastrowid
            
            cursor.execute("""
                SELECT id FROM effects WHERE description = ?
                """,(side_effect.lower(),))
            side_effect_id = cursor.fetchone()
            if side_effect_id:
                side_effect_id = side_effect_id[0]
            else:
                cursor.execute("""
                    INSERT INTO effects (description) VALUES (?)
                    """,(side_effect.lower(),))
                conn.commit()
                side_effect_id = cursor.lastrowid
            
            cursor.execute("""
                INSERT INTO properties (ingredient_id,effect_id,is_main) VALUES (?,?,?)
                """,(current_ingredient_id,main_effect_id,True))
            conn.commit()
            
            cursor.execute("""
                INSERT INTO properties (ingredient_id,effect_id,is_main) VALUES (?,?,?)
                """,(current_ingredient_id,side_effect_id,False))
            conn.commit()
                


In [10]:
isinstance(row['title'],str)

False